In [ ]:
# Suggested Visualizations for Tableau Dashboard

# Geographic Distribution:
# Create a choropleth map showing the distribution of billionaires across different countries. This can highlight where the wealthiest individuals are concentrated globally.

# Industry Breakdown:
# Use a bar chart or treemap to visualize which industries are the most common among billionaires. It can provide insight into which sectors have the most wealth.

# Wealth Distribution:
# A histogram or boxplot can be used to show the distribution of net worth. This helps visualize the range of wealth, from the median to outliers.

# Gender Diversity:
# A pie chart or bar chart can illustrate the gender distribution among billionaires. This could be an insightful metric regarding gender diversity in wealth.

# Source of Wealth:
# Create a stacked bar chart to compare the number of billionaires who are self-made versus those who inherited their wealth, possibly by country or industry.

# Age vs. Net Worth:
# A scatterplot can help show the relationship between age and net worth. It can also highlight the youngest billionaires and the wealthiest individuals.

# Net Worth Growth:
# If you have time-series data, a line chart can show changes in billionaires' net worth over time, revealing growth trends or declines.

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import os

# Unzipping the file
with zipfile.ZipFile('Billionaires.zip', 'r') as zip_ref:
    zip_ref.extractall('Billionaires')

# Get list of all files in the extracted folder
file_list = os.listdir('Billionaires')

# Load the first CSV file into a DataFrame as an example
csv_file = [file for file in file_list if file.endswith('.csv')][0]
df = pd.read_csv(f'Billionaires/{csv_file}')

# Display the first few rows to inspect the data
print(df.head())

   year  month  rank net_worth     last_name            first_name  \
0  1997      7   NaN     2.0 B  Sophonpanich                Chatri   
1  1997      7   NaN     1.8 B     Adulyadej         King Bhumibol   
2  1998      7   NaN     3.3 B         Safra                Edmond   
3  1999      7   NaN     1.0 B       Hinduja  Srichand, Gopichand,   
4  1999      7   NaN     7.1 B       Schwarz                Dieter   

                      full_name  birth_date   age gender  \
0  Chatri Sophonpanich & family  1934-02-28  73.0   Male   
1       King Bhumibol Adulyadej  1927-12-05  69.0   Male   
2                  Edmond Safra  1932-08-06  65.0   Male   
3  Srichand & Gopichand Hinduja  1935-11-28   NaN    NaN   
4                Dieter Schwarz  1939-09-24  59.0   Male   

  country_of_citizenship country_of_residence city_of_residence  \
0               Thailand             Thailand           Bangkok   
1               Thailand                  NaN               NaN   
2                

C:\Users\milto\AppData\Local\Temp\ipykernel_22076\1827356906.py:17: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'Billionaires/{csv_file}')


In [11]:
# Inital Exploration and Cleaniing

# Check data types and non-null counts
print("\nData Types and Non-Null Counts:\n")
print(df.info())

# Check for missing values in each column
print("\nMissing Values:\n")
print(df.isnull().sum())

# View basic statistics of numerical columns
print("\nSummary Statistics:\n")
print(df.describe())

# Display unique values of key categorical columns (e.g., 'Country', 'Industry')
categorical_columns = df.select_dtypes(include='object').columns
for col in categorical_columns:
    print(f"\nUnique values in '{col}':")
    print(df[col].unique())



Data Types and Non-Null Counts:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34511 entries, 0 to 34510
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   year                      34511 non-null  int64  
 1   month                     34511 non-null  int64  
 2   rank                      34493 non-null  float64
 3   net_worth                 34511 non-null  object 
 4   last_name                 30822 non-null  object 
 5   first_name                30760 non-null  object 
 6   full_name                 34511 non-null  object 
 7   birth_date                30044 non-null  object 
 8   age                       33765 non-null  float64
 9   gender                    30682 non-null  object 
 10  country_of_citizenship    34502 non-null  object 
 11  country_of_residence      33792 non-null  object 
 12  city_of_residence         33514 non-null  object 
 13  business_category         2

In [ ]:
# Interpretation

1. Handle Missing Values

    Highly Sparse Columns: Columns like organization_name and position_in_organization have many missing values (only ~4,718 and ~5,044 non-null values, respectively, out of 34,511). You could consider dropping these columns if they do not provide significant value for your analysis.
    Other Columns:
        Fill Missing Values:
            For rank and age, you can fill the missing values with the mean or median.
            For birth_date, first_name, last_name, etc., you can decide whether to fill with placeholders like "Unknown" or drop rows depending on their importance.
        Drop Rows: For critical columns like country_of_citizenship or business_category, if only a small number of rows are missing, you can drop those rows.

2. Convert Data Types

    The net_worth column is currently of type object, and it also contains a suffix (B for billions). You should clean this column to remove the suffix and convert it to a numeric type for further analysis.

3. Remove Duplicates

    Check and remove any duplicate rows.

4. Feature Engineering

    Consider creating a new feature for easier analysis. For example, you could create an age group feature to categorize billionaires into different age brackets.

In [13]:
# Handle Missing Values

# Drop columns with too many missing values
df.drop(columns=['organization_name', 'position_in_organization'], inplace=True)

# Fill missing values in rank and age with median values
df['rank'].fillna(df['rank'].median(), inplace=True)
df['age'].fillna(df['age'].median(), inplace=True)

# Fill categorical columns with 'Unknown'
categorical_columns = ['first_name', 'last_name', 'birth_date', 'gender', 'country_of_residence', 'business_category']
for col in categorical_columns:
    df[col].fillna('Unknown', inplace=True)

In [15]:
# Convert 'net_worth' to a numeric column by removing 'B' and converting to float
df['net_worth'] = df['net_worth'].str.replace(' B', '').astype(float)

In [17]:
# Check for duplicates
print(f"Number of duplicate rows: {df.duplicated().sum()}")

# Drop duplicate rows
df.drop_duplicates(inplace=True)

Number of duplicate rows: 1


In [19]:
# Create age group categories
bins = [0, 30, 50, 70, 100, df['age'].max()]
labels = ['<30', '30-50', '50-70', '70-100', '>100']
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels)

# Check distribution of age groups
print(df['age_group'].value_counts())

age_group
50-70     18238
70-100    10340
30-50      5551
<30         168
>100          5
Name: count, dtype: int64


In [ ]:
# Suggested Visualizations for Tableau Dashboard

In [21]:
# Gender Distribution
# Visualization Type: Pie Chart or Bar Chart
# Details: Create a pie chart showing the proportion of male vs. female billionaires.

gender_distribution = df['gender'].value_counts()
print(gender_distribution)

gender
Male       27279
Unknown     3829
Female      3402
Name: count, dtype: int64


In [23]:
# Net Worth by Country
# Visualization Type: Bar Chart or Choropleth Map
# Details: Show the total or average net worth of billionaires per country.

net_worth_by_country = df.groupby('country_of_citizenship')['net_worth'].sum().reset_index()
net_worth_by_country.columns = ['Country', 'Total_Net_Worth']
print(net_worth_by_country)

          Country  Total_Net_Worth
0         Algeria            41.40
1          Angola            21.40
2       Argentina           185.05
3         Armenia             3.30
4       Australia          1733.80
..            ...              ...
85  United States         52909.90
86        Uruguay             5.20
87      Venezuela           175.40
88        Vietnam           108.90
89       Zimbabwe            13.20

[90 rows x 2 columns]


In [27]:
# Industry Representation
# Visualization Type: Bar Chart or Treemap
# Details: Visualize which industries have the most billionaires, using the business_category column.

industry_representation = df['business_category'].value_counts().reset_index()
industry_representation.columns = ['Industry', 'Number_of_Billionaires']
print(industry_representation)

                      Industry  Number_of_Billionaires
0                      Unknown                    5842
1                   Technology                    2949
2             Fashion & Retail                    2730
3                Manufacturing                    2715
4                  Real Estate                    2398
5                  Diversified                    2151
6        Finance & Investments                    1900
7      Finance and Investments                    1755
8                   Healthcare                    1726
9                       Energy                    1281
10           Food and Beverage                    1147
11             Food & Beverage                    1016
12       Media & Entertainment                     887
13                     Service                     884
14             Metals & Mining                     814
15                  Automotive                     670
16                 Investments                     663
17  Constr

In [25]:
# Net Worth Distribution by Age Group
# Visualization Type: Boxplot or Bar Chart
# Details: Show how net worth is distributed across different age groups. A boxplot could illustrate the median, quartiles, and outliers for each age group.

net_worth_by_age_group = df.groupby('age_group')['net_worth'].describe()
print(net_worth_by_age_group)

             count      mean       std  min     25%  50%    75%    max
age_group                                                             
<30          168.0  3.302083  4.903507  1.0  1.3875  1.9  3.125   34.7
30-50       5551.0  3.737354  7.637375  1.0  1.3000  1.9  3.300  219.0
50-70      18238.0  3.795427  6.935048  1.0  1.4000  2.1  3.700  195.0
70-100     10340.0  4.884732  8.819453  0.6  1.6000  2.6  4.700  233.0
>100           5.0  2.340000  1.395708  1.1  1.3000  1.7  3.300    4.3


C:\Users\milto\AppData\Local\Temp\ipykernel_22076\1468711751.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  net_worth_by_age_group = df.groupby('age_group')['net_worth'].describe()


In [29]:
# Net Worth vs. Age (Scatterplot)
# Visualization Type: Scatterplot
# Details: Plot net worth against age to understand any potential trends between age and wealth.

scatter_data = df[['age', 'net_worth']]
print(scatter_data)

        age  net_worth
0      73.0        2.0
1      69.0        1.8
2      65.0        3.3
3      63.0        1.0
4      59.0        7.1
...     ...        ...
34506  63.0        1.0
34507  60.0        1.0
34508  66.0        1.0
34509  55.0        1.0
34510  60.0        1.0

[34510 rows x 2 columns]


In [31]:
# Self-Made vs. Inherited Wealth
# Visualization Type: Pie Chart or Bar Chart
# Details: Visualize the number of billionaires who are self-made versus those who inherited their wealth.

self_made_distribution = df['self_made'].value_counts()
print(self_made_distribution)

self_made
True     19468
False    11353
Name: count, dtype: int64


In [33]:
# Age Group Distribution
# Visualization Type: Bar Chart
# Details: Display the count of billionaires in each age group.

print(df['age_group'].value_counts())

age_group
50-70     18238
70-100    10340
30-50      5551
<30         168
>100          5
Name: count, dtype: int64
